In [6]:
import re
from datetime import datetime

_MONTHS_ES = {
    "ene": 1, "enero": 1,
    "feb": 2, "febrero": 2,
    "mar": 3, "marzo": 3,
    "abr": 4, "abril": 4,
    "may": 5, "mayo": 5,
    "jun": 6, "junio": 6,
    "jul": 7, "julio": 7,
    "ago": 8, "agosto": 8,
    # septiembre tiene variantes en ES/LA
    "sep": 9, "sept": 9, "septiembre": 9, "set": 9, "setiembre": 9,
    "oct": 10, "octubre": 10,
    "nov": 11, "noviembre": 11,
    "dic": 12, "diciembre": 12,
}

# día (1 o 2 dígitos) + opcional "de" + mes (abr./completo) + opcional "de" + año (4 dígitos)
# admite puntos en abreviaturas (ej. "sept."), espacios múltiples, y texto alrededor.
_DATE_RE = re.compile(
    r"(?P<d>\d{1,2})\s*(?:de\s+)?"
    r"(?P<m>ene(?:ro)?|feb(?:rero)?|mar(?:zo)?|abr(?:il)?|may(?:o)?|jun(?:io)?|jul(?:io)?|ago(?:sto)?|"
    r"sept(?:iembre)?|sep|set(?:iembre)?|oct(?:ubre)?|nov(?:iembre)?|dic(?:iembre)?)\.?\s*"
    r"(?:de\s+)?(?P<y>\d{4})",
    flags=re.IGNORECASE
)

In [9]:
import re
from datetime import datetime

_MONTHS_ES = {
    "ene": 1, "enero": 1,
    "feb": 2, "febrero": 2,
    "mar": 3, "marzo": 3,
    "abr": 4, "abril": 4,
    "may": 5, "mayo": 5,
    "jun": 6, "junio": 6,
    "jul": 7, "julio": 7,
    "ago": 8, "agosto": 8,
    # septiembre tiene variantes en ES/LA
    "sep": 9, "sept": 9, "septiembre": 9, "set": 9, "setiembre": 9,
    "oct": 10, "octubre": 10,
    "nov": 11, "noviembre": 11,
    "dic": 12, "diciembre": 12,
}

# día (1 o 2 dígitos) + opcional "de" + mes (abr./completo) + opcional "de" + año (4 dígitos)
# admite puntos en abreviaturas (ej. "sept."), espacios múltiples, y texto alrededor.
_DATE_RE = re.compile(
    r"(?P<d>\d{1,2})\s*(?:(?:de|del)\s+)?"
    r"(?P<m>ene(?:ro)?|feb(?:rero)?|mar(?:zo)?|abr(?:il)?|may(?:o)?|jun(?:io)?|jul(?:io)?|ago(?:sto)?|"
    r"sept(?:iembre)?|sep|set(?:iembre)?|oct(?:ubre)?|nov(?:iembre)?|dic(?:iembre)?)\.?\s*"
    r"(?:(?:de|del)\s+)?(?P<y>\d{4})",
    flags=re.IGNORECASE
)

def extract_spanish_date_ddmmyyyy(text: str) -> str | None:
    """
    Extrae la primera fecha en español tipo '30 de mayo de 2025' de un texto
    y la devuelve como '30/05/2025'. Si no encuentra una válida, retorna None.
    """
    match = _DATE_RE.search(text)
    if not match:
        return None

    day = int(match.group("d"))
    month_key = match.group("m").lower().rstrip(".")
    # normaliza 'sept' y similares a claves del diccionario
    if month_key not in _MONTHS_ES:
        # Por si llegara algo raro tipo 'set.' o 'sept.'
        month_key = month_key.rstrip(".")
    month = _MONTHS_ES.get(month_key)
    year = int(match.group("y"))
    if not month:
        return None

    # valida la fecha (ej. 31/02 no pasa)
    try:
        dt = datetime(year, month, day)
    except ValueError:
        return None

    return dt.strftime("%d/%m/%Y")

# --- Ejemplos de uso ---
examples = [
    "San Isidro, 15 de mayo del 2023.",
    "San Isidrio 30 de mayo de 2025",
    "Contrato firmado el 1 de ene. de 2024 en Lima",
    "Vence: 7 setiembre 2023",
    "Pago: 15   dic  2022",
    "Reunión 31/02/2024 (texto confuso) y 28 de febrero de 2024",  # solo tomará la válida en formato español
]

for s in examples:
    print(s, "->", extract_spanish_date_ddmmyyyy(s))


San Isidro, 15 de mayo del 2023. -> 15/05/2023
San Isidrio 30 de mayo de 2025 -> 30/05/2025
Contrato firmado el 1 de ene. de 2024 en Lima -> 01/01/2024
Vence: 7 setiembre 2023 -> 07/09/2023
Pago: 15   dic  2022 -> 15/12/2022
Reunión 31/02/2024 (texto confuso) y 28 de febrero de 2024 -> 28/02/2024


In [3]:
import pandas as pd

df = pd.DataFrame([
    {"disbursed_amount": 1000.5, "reduced_amount": 200.0, "amount": 800.5},
    {"disbursed_amount": 500.0, "reduced_amount": 0.0, "amount": 500.0},
])

In [4]:
from pydantic import BaseModel

class FinancialRecord(BaseModel):
    disbursed_amount: float
    reduced_amount: float
    amount: float

In [5]:
if df is None or df.empty:
    raise ValueError("El DataFrame está vacío")

# Convierte la primera fila a dict
first_row = df.iloc[0].to_dict()

# Crea el modelo
record = FinancialRecord(**first_row)

print(record)

disbursed_amount=1000.5 reduced_amount=200.0 amount=800.5
